<a href="https://colab.research.google.com/github/ShibinMohammed/Data-science-projects/blob/main/Predicting_a_pulsar_star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting A Pulsar Star

### Overview

Pulsar stars (https://www.youtube.com/watch?v=gjLk_72V9Bw) are a rare type of Neutron stars (https://www.youtube.com/watch?v=hCwDNXKlN8Q) that produce radio emissions detectable on Earth. They are of considerable scientific interest as probes of space-time, the interstellar medium, and states of matter.

You can learn more about the Pulsar and neutron star by watching this video: https://www.youtube.com/watch?v=RrMvUL8HFlM

As pulsars rotate, their emission beam sweeps across the sky, and when this crosses our line of sight, it produces a detectable pattern of broadband radio emission.

<img src='https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/images/lightcurve_pulsar.gif'>

As pulsars rotate rapidly, this pattern repeats periodically. Thus pulsar search involves looking for periodic radio signals using large radio telescopes.

Each pulsar produces a slightly different emission pattern, which varies slightly with each rotation. Thus a potential signal detection known as a 'candidate', is averaged over many rotations of the pulsar, as determined by the length of observation. In the absence of additional info, each candidate could potentially describe a real pulsar. However, in practice almost all detections are caused by Radio Frequency Interference (RFI) and noise, making legitimate signals hard to find.

Machine learning tools are now being used to automatically label pulsar candidates to facilitate rapid analysis. The classification algorithms, in particular, are being widely adopted, which treat the candidate datasets as binary classification problems (predict either `0` or `1`). Here, the legitimate pulsar examples are a minority positive class (less in numbers), and the remaining examples are a majority negative class.

The class labels used are `0` (negative class) and `1` (positive class). Hence, **we need to deploy the XGBoost Classifier classification model which can accurately detect the class 1 examples.**


### Attribute Information

Each candidate is described by 8 continuous variables and a single class variable. The first four are simple statistics obtained from the integrated pulse profile (folded profile). This is an array of continuous variables that describe a longitude-resolved version of the signal that has been averaged in both time and frequency. The remaining four variables are similarly obtained from the DM-SNR curve. These 8 variables are summarised below:

1. The mean of the integrated profile.

2. The standard deviation of the integrated profile.

3. Excess kurtosis of the integrated profile.

4. The skewness of the integrated profile.

5. The mean of the DM-SNR curve.

6. The standard deviation of the DM-SNR curve.

7. Excess kurtosis of the DM-SNR curve.

8. The skewness of the DM-SNR curve.

Source: https://archive.ics.uci.edu/ml/datasets/HTRU2

**Courtesy**

Dr Robert Lyon

University of Manchester

School of Physics and Astronomy

Alan Turing Building

Manchester M13 9PL

United Kingdom

robert.lyon@manchester.ac.uk

#### Dataset Links

You can get the datasets from the following links:

1. Train dataset: https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/project-5/pulsar-star-prediction-train.csv

2. Test dataset: https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/project-5/pulsar-star-prediction-test.csv


In [ ]:
# Create the DataFrames for both the train and test datasets and store them in the 'train_df' and 'test_df' variables respectively.
import pandas as pd
import numpy as np

train_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/project-5/pulsar-star-prediction-train.csv')
test_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/project-5/pulsar-star-prediction-test.csv')

In [ ]:
# Display the first 5 rows of the 'train_df' DataFrame.
train_df.head()

,target_class,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,0,111.109375,53.131064,0.280253,-0.222447,3.011706,20.355820,7.585482,62.383270
1,0,151.945312,47.973350,-0.250834,0.275367,2.115385,14.195484,11.640297,173.592172
2,1,52.335938,34.775008,2.478375,10.179171,8.230769,34.775947,4.652342,21.987882
3,0,121.562500,48.569498,-0.033391,-0.323514,2.595318,15.089924,8.734079,98.584122
4,0,133.664062,59.137852,-0.164198,-0.552877,1.542642,12.052034,12.226623,196.522501


In [ ]:
# Display the first 5 rows of the 'test_df' DataFrame.
test_df.head()

,target_class,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,0,116.906250,48.920605,0.186046,-0.129815,3.037625,17.737102,8.122621,78.813405
1,1,75.585938,34.386254,2.025498,8.652913,3.765050,21.897049,7.048189,55.878791
2,0,103.273438,46.996628,0.504295,0.821088,2.244983,15.622566,9.330498,105.134941
3,1,101.078125,48.587487,1.011427,1.151870,81.887960,81.464136,0.485105,-1.117904
4,0,113.226562,48.608804,0.291538,0.292120,6.291806,26.585056,4.540138,21.708268


In [ ]:
# Display the last 5 rows of the 'train_df' DataFrame.
train_df.tail()

,target_class,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
11986,0,124.312500,53.179053,-0.012418,-0.556021,7.186455,29.308266,4.531382,21.725143
11987,0,115.617188,46.784600,0.218177,0.226757,6.140468,30.271961,5.732201,34.357283
11988,0,116.031250,43.213846,0.663456,0.433088,0.785117,11.628149,17.055215,312.204325
11989,0,135.664062,49.933749,-0.089940,-0.226726,3.859532,21.501505,7.398395,62.334018
11990,0,120.726562,50.472256,0.346178,0.184797,0.769231,11.792603,17.662222,329.548016


In [ ]:
# Display the last 5 rows of the 'test_df' DataFrame.
test_df.tail()

,target_class,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
5902,0,120.140625,45.794673,0.466357,0.528349,2.286789,16.909127,9.823143,111.061145
5903,0,145.445312,45.821803,-0.175460,0.401138,3.040970,16.063758,7.755287,80.304937
5904,0,117.875000,51.089535,-0.091320,-0.268881,2.075251,12.376724,10.736269,167.163044
5905,0,126.812500,46.832606,0.291279,0.183496,1.045151,9.514711,17.821202,431.240745
5906,0,154.835938,46.133938,-0.313312,0.082159,7.258361,30.115319,5.232192,30.444595


In [ ]:
# Print the number of rows and columns in both the DataFrames.
print(train_df.shape)
print(test_df.shape)

(11991, 9)
(5907, 9)


In [ ]:
# Check for the missing values in the 'train_df' DataFrame.
train_df.isna().sum()

target_class                                     0
 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
dtype: int64

In [ ]:
# Check for the missing values in the 'test_df' DataFrame.
test_df.isna().sum()

target_class                                     0
 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
dtype: int64

In [ ]:
# Print the count of the '0' and '1' classes in the 'train_df' DataFrame.
print(train_df['target_class'].value_counts())

0    10878
1     1113
Name: target_class, dtype: int64


In [ ]:
# Print the count of the '0' and '1' classes in the 'test_df' DataFrame.
print(test_df['target_class'].value_counts())

0    5381
1     526
Name: target_class, dtype: int64


In [ ]:
# Get the feature variables from the 'train_df' DataFrame.
x_train = train_df.iloc[:,1:]
# Display the first 5 rows of the 'x_train' DataFrame.
x_train.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,111.109375,53.131064,0.280253,-0.222447,3.011706,20.355820,7.585482,62.383270
1,151.945312,47.973350,-0.250834,0.275367,2.115385,14.195484,11.640297,173.592172
2,52.335938,34.775008,2.478375,10.179171,8.230769,34.775947,4.652342,21.987882
3,121.562500,48.569498,-0.033391,-0.323514,2.595318,15.089924,8.734079,98.584122
4,133.664062,59.137852,-0.164198,-0.552877,1.542642,12.052034,12.226623,196.522501


In [ ]:
# Get the feature variables from the 'test_df' DataFrame.
x_test = test_df.iloc[:,1:]
# Display the first 5 rows of the 'x_test' DataFrame.
x_test.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,116.906250,48.920605,0.186046,-0.129815,3.037625,17.737102,8.122621,78.813405
1,75.585938,34.386254,2.025498,8.652913,3.765050,21.897049,7.048189,55.878791
2,103.273438,46.996628,0.504295,0.821088,2.244983,15.622566,9.330498,105.134941
3,101.078125,48.587487,1.011427,1.151870,81.887960,81.464136,0.485105,-1.117904
4,113.226562,48.608804,0.291538,0.292120,6.291806,26.585056,4.540138,21.708268


In [ ]:
# Get the target variable from the 'train_df' DataFrame.
y_train = train_df.iloc[:,0]
# Display the first 5 rows of the 'y_train' Pandas series.
y_train.head()

0    0
1    0
2    1
3    0
4    0
Name: target_class, dtype: int64

In [ ]:
# Get the target variable from the 'test_df' DataFrame.
y_test = test_df.iloc[:,0]
# Display the first 5 rows of the 'y_test' Pandas series.
y_test.head()

0    0
1    1
2    0
3    1
4    0
Name: target_class, dtype: int64

In [ ]:
# Build A XGBoost Classifier model
# Import the xgboost module.
import xgboost as xg
# Import the confusion_matrix and classification_report modules.
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
# Predict the target variable based on the feature variables of the test dataframe.
XGB = xg.XGBClassifier()
XGB.fit(x_train,y_train)
pred = XGB.predict(x_test)
pred

array([0, 1, 0, ..., 0, 0, 0])

In [ ]:
# Print the confusion matrix to see the number of TN, FN, TP and FP.
print(confusion_matrix(y_test,pred))

[[5348   33]
 [  82  444]]


In [ ]:
# Print the precision, recall and f1-score values for both the '0' and '1' classes.
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5381
           1       0.93      0.84      0.89       526

    accuracy                           0.98      5907
   macro avg       0.96      0.92      0.94      5907
weighted avg       0.98      0.98      0.98      5907

